# IBM Developer Skills Network

## Access Db2 on cloud using Python

In [26]:
import ibm_db
import pandas as pd

# Get the user id, passwd for IBM Db2 instance on cloud

In [27]:
path = 'G:\Google Drive\My learning courses\IBM Db2 connection.csv'
df = pd.read_csv(path)

In [28]:
dsn_hostname = df.value[0]
dsn_uid = df.value[1]
dsn_pwd = df.value[2]

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_port = "50000"                # e.g. "50000" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"

# Create the DB2 database connection

In [29]:
#Create the dsn connection string
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

In [30]:
# Create database connection
try:
    conn = ibm_db.connect(dsn, '', '')
    print('Connected to database!')
except:
    print ('Unable to connect: ', ibm_db.conn_errormsg())

Connected to database!


# Create a table in the database

In [31]:
dropQuery = 'drop table INSTRUCTOR'

# Delete the table if it was already created before
try:
    dropStmt = ibm_db.exec_immediate(conn, dropQuery)
    print('Success drop table INSTRUCTOR')
except:
    print('Table INSTRUCTOR not existed!')

# Create the INSTRUCTOR table
createQuery = "create table INSTRUCTOR(ID INTEGER PRIMARY KEY NOT NULL, FNAME VARCHAR(20), LNAME VARCHAR(20), CITY VARCHAR(20), CCODE CHAR(2))"

createStmt = ibm_db.exec_immediate(conn, createQuery)
print('Table INSTRUCTOR Created!')

Success drop table INSTRUCTOR
Table INSTRUCTOR Created!


In [32]:
# Insert data into table
insertQuery = "insert into INSTRUCTOR values (1, 'Rav', 'Ahuja', 'TORONTO', 'CA')"

#execute the insert statement
insertStmt = ibm_db.exec_immediate(conn, insertQuery)

In [33]:
# Insert the next 2 data rows
insertQuery2 = "insert into INSTRUCTOR values (2, 'Raul', 'Chong', 'Markham', 'CA'), (3, 'Hima', 'Vasudevan', 'Chicago', 'US')"

#execute the statement
insertStmt2 = ibm_db.exec_immediate(conn, insertQuery2)

# Query data in the table

In [34]:
selectQuery = 'select * from INSTRUCTOR'
selectStmt = ibm_db.exec_immediate(conn, selectQuery)
ibm_db.fetch_both(selectStmt) # 1st row

{'ID': 1,
 0: 1,
 'FNAME': 'Rav',
 1: 'Rav',
 'LNAME': 'Ahuja',
 2: 'Ahuja',
 'CITY': 'TORONTO',
 3: 'TORONTO',
 'CCODE': 'CA',
 4: 'CA'}

In [35]:
#Fetch the rest of the rows and print the ID and FNAME for those rows
while ibm_db.fetch_row(selectStmt) != False:
   print (" ID:",  ibm_db.result(selectStmt, 0), " FNAME:",  ibm_db.result(selectStmt, "FNAME"))

 ID: 2  FNAME: Raul
 ID: 3  FNAME: Hima


## Update statement

In [36]:
updateQuery = "update INSTRUCTOR set CITY='MOOSETOWN' where FNAME='Rav'"
updateStmt = ibm_db.exec_immediate(conn, updateQuery)

# Retrieve data into Pandas

In [37]:
import ibm_db_dbi

pconn = ibm_db_dbi.Connection(conn)

In [38]:
selectQuery = 'select * from INSTRUCTOR'

# to pandas frame
pdf = pd.read_sql(selectQuery, pconn)

pdf.LNAME[0]

'Ahuja'

In [39]:
pdf

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,MOOSETOWN,CA
1,2,Raul,Chong,Markham,CA
2,3,Hima,Vasudevan,Chicago,US


In [40]:
pdf.shape

(3, 5)

## Select table from table name

In [43]:
selectQuery = "select * from SYSCAT.TABLES where TABNAME = 'SCHOOLS'"

# to pandas frame
pdf = pd.read_sql(selectQuery, pconn)

pdf.head()

,TABSCHEMA,TABNAME,OWNER,OWNERTYPE,TYPE,STATUS,BASE_TABSCHEMA,BASE_TABNAME,ROWTYPESCHEMA,ROWTYPENAME,...,ONCOMMIT,LOGGED,ONROLLBACK,LASTUSED,CONTROL,TEMPORALTYPE,TABLEORG,EXTENDED_ROW_SIZE,PCTEXTENDEDROWS,REMARKS
0,WRD87479,SCHOOLS,WRD87479,U,T,N,None,None,None,None,...,,,,2021-05-03,,N,R,N,-1.0,None


## Check number of Elementary school

In [44]:
selectQuery = "select count(*) from SCHOOLS where school_type = 'ES'"
# to pandas frame
pdf = pd.read_sql(selectQuery, pconn)

pdf.head()

,1
0,462.0


## Check highest safety score

In [45]:
selectQuery = "select MAX(SAFETY_SCORE) from SCHOOLS"
# to pandas frame
pdf = pd.read_sql(selectQuery, pconn)

pdf.head()

,1
0,99


## Which schools have the highest safety score

In [46]:
selectQuery = "select NAME_OF_SCHOOL from SCHOOLS where Safety_Score = (select MAX(SAFETY_SCORE) from SCHOOLS)"
# to pandas frame
pdf = pd.read_sql(selectQuery, pconn)

pdf.head()

,NAME_OF_SCHOOL
0,Abraham Lincoln Elementary School
1,Alexander Graham Bell Elementary School
2,Annie Keller Elementary Gifted Magnet School
3,Augustus H Burley Elementary School
4,Edgar Allan Poe Elementary Classical School


## Top 10 schools with the highest Average student attendance

In [51]:
selectQuery = "select NAME_OF_SCHOOL from SCHOOLS order by AVERAGE_STUDENT_ATTENDANCE Desc limit 10"
# to pandas frame
pdf = pd.read_sql(selectQuery, pconn)

pdf.head(10)

,NAME_OF_SCHOOL
0,Velma F Thomas Early Childhood Center
1,John Charles Haines Elementary School
2,James Ward Elementary School
3,Edgar Allan Poe Elementary Classical School
4,Rachel Carson Elementary School
5,Orozco Fine Arts & Sciences Elementary School
6,Annie Keller Elementary Gifted Magnet School
7,Andrew Jackson Elementary Language Academy
8,Lenart Elementary Regional Gifted Center
9,Disney II Magnet School


## Retrieve list of 5 schools with the lowest Average Student Attendance sorted in ascending order based on attendance

In [50]:
selectQuery = "select NAME_OF_SCHOOL, Average_Student_Attendance from SCHOOLS order by AVERAGE_STUDENT_ATTENDANCE limit 5"
# to pandas frame
pdf = pd.read_sql(selectQuery, pconn)

pdf.head()

,NAME_OF_SCHOOL,AVERAGE_STUDENT_ATTENDANCE
0,Richard T Crane Technical Preparatory High School,57.90%
1,Barbara Vick Early Childhood & Family Center,60.90%
2,Dyett High School,62.50%
3,Wendell Phillips Academy High School,63.00%
4,Orr Academy High School,66.30%


## Remove the % sign from above result

In [59]:
selectQuery = "select NAME_OF_SCHOOL, Replace(Average_Student_Attendance, '%', '') As Ave_Stu from SCHOOLS order by AVERAGE_STUDENT_ATTENDANCE limit 5"
# to pandas frame
pdf = pd.read_sql(selectQuery, pconn)

pdf.head()

,NAME_OF_SCHOOL,AVE_STU
0,Richard T Crane Technical Preparatory High School,57.90
1,Barbara Vick Early Childhood & Family Center,60.90
2,Dyett High School,62.50
3,Wendell Phillips Academy High School,63.00
4,Orr Academy High School,66.30


## Which shcools have Average Student Attendance lower than 70%

In [62]:
selectQuery = "select NAME_OF_SCHOOL, Replace(Average_Student_Attendance, '%', '') As Ave_Stu from SCHOOLS where Replace(Average_Student_Attendance, '%', '') < 70"
# to pandas frame
pdf = pd.read_sql(selectQuery, pconn)

pdf.head()

,NAME_OF_SCHOOL,AVE_STU
0,Barbara Vick Early Childhood & Family Center,60.90
1,Chicago Vocational Career Academy High School,68.80
2,Dyett High School,62.50
3,Manley Career Academy High School,66.80
4,Orr Academy High School,66.30


## Get the total College Enrollment for each Community Area

In [64]:
selectQuery = "select COMMUNITY_AREA_NAME, sum(COLLEGE_ENROLLMENT) as Col_En_sum from SCHOOLS group by COMMUNITY_AREA_NAME"
# to pandas frame
pdf = pd.read_sql(selectQuery, pconn)

pdf.head()

,COMMUNITY_AREA_NAME,COL_EN_SUM
0,ALBANY PARK,6864
1,ARCHER HEIGHTS,4823
2,ARMOUR SQUARE,1458
3,ASHBURN,6483
4,AUBURN GRESHAM,4175


## Get the 5 community Areas with the least total College Enrollment sorted in ascending order

In [65]:
selectQuery = "select COMMUNITY_AREA_NAME, sum(COLLEGE_ENROLLMENT) as total_enrollment from SCHOOLS group by COMMUNITY_AREA_NAME order by total_enrollment asc limit 5"
# to pandas frame
pdf = pd.read_sql(selectQuery, pconn)

pdf.head()

,COMMUNITY_AREA_NAME,TOTAL_ENROLLMENT
0,OAKLAND,140
1,FULLER PARK,531
2,BURNSIDE,549
3,OHARE,786
4,LOOP,871


# Close the connection

In [21]:
ibm_db.close(conn)

True